# 1. 베이지안 최적화 개요

## 블랙박스 최적화 문제
블랙박스 최적화 문제는 목적 함수, 제약식, 결정 변수 간 자세한 관계를 알 수 없는 최적화 문제를 말합니다. 구체적으로 추가적인 실험 및 측정 없이 결정변수에 따른 목적 함숫값을 모르거나 제약식을 만족하는지 알 수 없는 문제를 의미합니다.

## 블랙박스 최적화 문제의 해법
블랙박스 최적화 문제는 목적 함수를 정확히 알 수 없으므로 경험적으로 해결해야 합니다. 여기서 경험적으로 해결한다는 말은 데이터를 바탕으로 목적함수를 추론해가면서 탐색 공간을 결정한다는 의미입니다.

## 블랙박스 최적화 문제: 데이터 획득의 어려움
데이터 포인트가 많으면 많을수록 정확한 함수 추정이 가능합니다. 그러나 블랙박스 최적화 문제의 목적 함수를 평가하는 데 시간이 오래 걸려 많은
데이터 포인트를 얻기가 어렵습니다.

- 예를 들어, 하이퍼파라미터 튜닝 문제라면 특정 하이퍼파라미터를 갖는 모델을 학습하고 평가해야만 하나의 데이터 포인트를 얻을 수 있음
- 심지어는 문제에 따라 금전적인 비용이 발생하기도 함. 예를 들어, 원료 배합을 최적화하는 문제라면 실제로 원료 배합을 평가해야 데이터 포인트를 얻을 수 있는데, 그 과정에서 금전적인 비용이 발생할 수 있음

## 베이지안 최적화가 다루는 문제
베이지안 최적화는 블랙박스 최적화 문제에 대한 대표적인 해법으로 목적 함수가 다음과 같은 특성을 가질 때 적합합니다.
- 함숫값이 특정 구간에서 연속적임
- 목적 함수를 평가하는 데 많은 비용이 발생함
- 도함수를 알 수 없어 내리막 경사법 등을 사용할 수 없음
- 해를 평가하는 과정에서 노이즈(noise)가 발생하지 안흔다고 가정할 수 있음

## 베이지안 최적화의 구성
베이지안 최적화는 샘플러(sampler), 대체 모델(surrogate model), 획득 함수(acquisition function)가 상호 작용하면서 해를 탐색합니다.

- (1) 샘플러를 이용해 초기 해 집단 X0 = 𝒙10,𝒙20,⋯,𝒙𝑛0 를 생성합니다. 여기서 X0은 0번째 이터레이션에서의 해 집단을, 𝒙𝑖0은 0번째로 만든 해 집단의 𝑖번째 해를 나타냅니다.
- (2) 목적 함수를 이용하여 해를 평가합니다. 
- (3) 종료 조건을 만족하는지 확인합니다. 만족한다면 현재까지 찾은 해 가운데 가장 좋은 해를 반환하고 알고리즘을 종료하며, 그렇지 않으면 (4)로 갑니다.
- (4) 현재까지 탐색한 해를 사용해 대체 모델을 학습하거나 업데이트합니다. 대체 모델은 특징이 해이고 라벨이 목적 함숫값인 지도 학습 모델입니다. 
- (5) 샘플러를 사용해 여러 개의 해를 임의로 생성합니다. 생성한 해를 대체 모델과 획득 함수로 평가하여 우수한 해를 추천한 뒤 (2)로 되돌아갑니다.


# 2. 대체 모델

## 대체 모델이란?
대체 모델은 목적 함수를 모사하는 회귀 모델입니다.

- 대체 모델이 필요한 이유는 목적 함수를 이용해 해를 평가하는데많은 시간과 비용이 들기 때문
- 모든 해를 목적 함수를 사용하여 평가하지 않고 대체 모델로평가한다음에 평가할 만한 가치가 있는 해만 선별하여 목적 함수를 사용하여평가함
- 대체 모델은 지도 학습 모델이므로 학습만되면 빠르게해를 평가할수있음
- 그러나 모든 지도 학습 모델이 그렇듯이 예측 오차가 없을 수는 없어, 열등한 해를 좋다고 평가하는 위험이 있음

대체 모델은 목적 함수를 사용해 해를 평가하는 시간을 단축하는 데 필요함

## 가우시안 프로세스
가우시안 프로세스(Gaussian Process)는 대체 모델로 가장 많이 사용되는 지도 학습 모델입니다. 그 이유는 가우시안 프로세스가 목적 함숫값의 분포를 추정하는 커널 기반의 비모수 모델 (non-parametric model) 1) 이기 때문입니다.
- 1) 파라미터 수가 학습 데이터 크기에 정비례하는 모델입니다. 예를 들어, 대표적인 모수 모델인 회귀 모델은 데이터 개수에 상관없이 파라미터인 계수와 절편의 개수가 일정합니다. 그러나 가우시안 프로세스와 k-최근접 이웃과 같은 비모수 모델은 학습 데이터가 늘어남에 따라 더정교해지지만 계산해야 하는 부분이 늘어납니다.

가우시안 프로세스를 많이 사용하는 이유
- 다섯 개 데이터 포인트로 추정할 수 있는 함수의 개수는 무한함
- 가우시안 프로세스는 다른 모델처럼 데이터 포인트를 가장 잘 설명 하는 특정한 함수를 찾는 것이 아니라, 데이터 포인트를 설명할 수 있는 함수의 분포를 추정함
• 따라서 각 해의 함숫값에 관한 신뢰 구간을 계산할 수있고, 그에 따라 실제 평가가 필요한 불확실한 해를 선별할 수있음
• 커널 기반의 비모수 모델이라는 것은 새로운 해의 목적 함숫값을 추정하는데 목적 함숫값을 알고 있는 기존 해를 활용한다는 뜻임
• 실제 목적 함숫값을 아는 해가 늘어날수록 더 자세한 추정을 할 수 있어 매 이터레이션 마다 업데이트 해야하는 대체 모델의 특성에 부합함


## 가우시안 프로세스 : 함수에 대한 확률 분포
가우시안 프로세스는 미지의 함수 𝒇에 대한 확률 분포라고 할 수 있습니다. 즉, 𝒙가 입력되면 𝑓 𝒙 를 출력하는 함수가 있는데, 𝑓 𝒙 가 하나로 정해지는 것 이 아니라 어떤 분포를 따르는 것입니다.

- 출력을 알고 있는 입력 -3, -2, 4에서는예측 구간의 길이가 0인데, 실제 출력을 알고 있으니 예측 편차가 없음
- 출력을 알고 있는 입력 근처의 값은 예측 구간이 짧고, 𝑥=1과같이 출력을 알고 있는 입력과 멀리 떨어져 있는 값 일수록 예측 구간이 매우 김
- 잘 아는 구간 일수록 편차가 적어 예측 구간이 짧지만, 잘 모르는 구간일수록 편차가 커서 예측 구간이 김
- 다시 말해, 관측한 데이터 X= 𝒙(1),𝒙(2),𝒙(3),⋯,𝒙(𝑛) 가 있을 때, 새로운 샘플 𝒙(𝑛+1)의 함숫값 𝑓 𝒙𝑛+1 는 확률 분포이며 이미 관측된 샘플이 𝒙𝑛+1 와 얼마나 가까운지에 따라 𝑓 𝒙𝑛+1 의 예측 구간의 길이가 결정됨

## 가우시안 프로세스 : 다변량 정규 분포
가우시안 프로세스는 함숫값이 다변량 정규 분포(multivariate normal distribution)를 따른다고 가정합니다. 다변량 정규 분포를 따르는 확률 변수는 단순히 정규 분포를 따르는 개별 확률 변수의 조합이 아니라, 서로의 분포에 영향을 줍니다.

- 다변량 정규 분포에 대해 깊은 이해가 중요한 것이 아니라, 각 변수가 서로 영향을 준다는 것이 중요함

가우시안 프로세스는 학습 데이터 𝑓 𝒙1:𝑛 = 𝑓 𝒙1 ,𝑓 𝒙2 ,⋯,𝑓 𝒙𝑛 와 새로 예측할 𝑓 𝒙𝑛+1 모두 같은 다변량 정규 분포를 따른다고 간주합니다.

## 가우시안 프로세스 : 학습
가우시안 프로세스 모델의 학습은 커널에 포함된 파라미터를 추정하는 것입니다.

## 가우시안 프로세스 : 문제점
가우시안 프로세스는 다변량 정규분포를 가정하기 때문에 몇 가지 문제점이 있습니다.

- Frazier1)에 따르면 20차원 이하의 실수 벡터에만 적합함
    -범주형 변수를 포함하는 모델 선택과 하이퍼파라미터 튜닝 문제에 가우시안 프로세스를 적용하는 것은 부적절합니다.
1) Frazier, P. I. (2018). A tutorial on Bayesian optimization. arXiv preprint arXiv:1807.02811
- 계층적 구조에 부적합함
    -예를 들어, 모델 선택과 하이퍼파라미터 튜닝 문제에서 모델 변수가 랜덤 포레스트면 튜닝해야 하는 하이퍼파라미터는 나무의 최대 깊이, 나무 개수 등 입니다. 그러나 모델 변수가 신경망이면 튜닝해야 하는 하이퍼파라미터는 은닉층의 구조가 됩니다. 이처럼 한 변수의 값에 따라 다른 변수를 사용하거나 사용하지 못하는 상황에서 가우시안 프로세스를 적용하기는 어렵습니다. 
- 어디까지나 적용했을 때 좋은 성능을 기대하기 어렵다는 이야기지 적용할 수 없다는 것은 아님
    -실제로 많은 연구에서 가우시안 프로세스를 연속형 변수로만 구성된 저차원의 데이터가 아닌 데이터에도 적용했습니다. 또한, 특수한 상황에 맞게 가우시안 프로세스 모델을 수정 합니다.


# 3. 샘플러와 대체 모델 실습

## 예제
샘플러와 대체 모델 실습을 진행할 예제는 다음과 같습니다.
- 목적 함수: maximize 𝑓 (𝑥_1,𝑥_2) = 𝑥1^2sin (5𝜋( −𝑥1 +2𝑥_2))
    - 목적 함수가 복잡하긴 하나, 사실 블랙박스 최적화 문제는 아님
- 제약식: 0≤𝑥_1,𝑥_2 ≤1

In [2]:
import numpy as np
def obj_func(x):
    return x[0]**2 * np.sin(5 * np.pi * (-x[0] + 2 * x[1]))

- 라인 2: 간결한 코드 작성을 위해 x1 과 x2를 따로 입력받지 않고 샘플 x를 입력 받습니다. x[0]를 x1으로, x[1]을 x2로 보면 됩니다.


## 샘플러 정의
x1과 x2가 모두 0과 1 사이의 값이므로 np.random.random 함수를 사용해 샘플러를 정의합니다.

샘플러 정의

In [ ]:
def sample(n):
    return np.random.random((n, 2))

- 라인 1-2: 샘플 개수 n을 입력받아 (n, 2) 크기의 난수 행렬을 반환합니다.

샘플링

In [ ]:
X = sample(100)
y = np.apply_along_axis(obj_func, axis = 1, arr = X)

- 라인 1: 샘플러를 사용해 100개의 해를 생성해 X에 저장합니다.
- 라인 2: X에 속한 모든 요소를 obj_func으로 평가합니다.

## 가우시안 프로세스
사이킷런의 GaussianProcessRegressor 클래스를 사용하면 가우시안 프로세스 모델을 만들 수 있습니다. 이렇게 만든 가우시안 프로세스 모델은 다른 사이킷런의 인스턴스와 마찬가지로 predict 메서드를 이용해 라벨을 예측할 수 있습니다. 차이가 있다면 가우시안 프로세스 모델은 return_std와 return_cov라는 인자로 표준편차와 다른 샘플과의 공분산을 출력할 수 있다는 점입니다.

주요 인자
- 인자
    - kernel : 가우시안 프로세스의 커널 함수를 지정합니다. sklearn.gausiian_process.kernels에 있는 커널 함수를 조합하여 설정할 수 있습니다.

가우시안 프로세스 모델 학습 예제

In [1]:
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
model = GPR(kernel = RBF() + WhiteKernel(), random_state=2022).fit(X, y)

NameError: name 'X' is not defined

- 라인 3: RBF 커널과 노이즈를 나타내는 WhiteKernel을 더한 커널을 갖는 가우시안 프로세스 모델을 X와 y로 학습합니다.

# 4. 획득 함수

## 획득 함수란?
획득 함수는 실제 출력을 모르는 해집합 𝑋new에 속한 모든 해를 평가하고 그 가운데 가장 값이 큰 해를 추천합니다. 상황에 따라서는 상위 𝑘개의 해를 추천 하기도 합니다.

𝒙_new∗ = argmax g(x) ( x ∈ x_new )

- 𝑔: 획득 함수
- 추천된 해 𝒙new∗ 는 실제로 평가를 진행하여 𝒙new∗ ,𝑓 𝒙new∗ 를 포함하여 가우시안 프로세스 모델을 재학습함


## 획득 함수의 전략
획득 함수는 착취(exploitation)와 탐색(exploration) 전략으로 구성됩니다. 착취 전략이란 좋은 해 근처의 해를 찾는 전략을, 탐색 전략이란 아직 탐색되지 않은 구간을 중심으로 찾는 전략을 의미합니다.

- 목적 함수
    - 현재까지 세 개의 입력 𝒙1 , 𝒙𝟐 , 𝒙𝟑 을 평가했음
    - 빨간 실선: 추정한 목적 함수의 평균
    - 주황색 점선: 추정한 목적 함수의 예측 구간

- 획득 함수
    - 𝑓 𝒙1 , 𝑓 𝒙2 , 𝑓 𝒙3 에 대한 편차는 0임
    - 𝒙1 , 𝒙𝟐 , 𝒙𝟑 에 대한 출력을 알고 있으므로, 평가할 가치가 없어 획득 함숫값이 0임
    - 편차가 크고 상대적으로 값도 크리라 예상되는 𝒙𝟐 와 𝒙𝟑 사이에 있는 값의 획득 함숫값이 다른 구간에 있는 값에 대한 획득 함숫값보다 큽니다.

## 개선 확률 함수
개선 확률 함수는 현재까지 찾은 최대 출력값보다 더 큰 함숫값을 도출할 확률입니다.

## 개선 확률 함수 실습
파이썬으로 개선 확률 함수를 구현해보겠습니다.

파이썬에서의 개선 확률 함수 예시

In [ ]:
from scipy.stats import norm
def POI(X_new, model, best_mu):
    mu, sigma = model.predict(X_new, return_std = True)
    output = norm.cdf((mu - best_mu)/(Sigma + 0.00000000001))
    return output

- 라인2: 새로운 입력 X_new와가우시안프로세스 모델인 model, 현재까지 가장 큰 평균인 best_mu를입력으로 하는 개선 확률 함수 POI를 정의합니다.
- 라인3: return_std 인자를 True로 설정해서 새로운 입력에 대한 평균과 표준편차를 계산하고 그 값을 mu와 sigma에 저장합니다.
- 라인4: 표준 정규 분포를 따르는 변수에 대해 (mu-best_mu)/(sigma + 0.0000000001)보다작을 확률을 output에 저장합니다. 이때, 분모가 0이 될 수 있으므로 분모에 매우 작은 양수를 더했습니다.

개선 확률 함수를 이용한 해 추천 예시

In [ ]:
best_mu = max(model.predict(X))
X_new = sampler(100)
score_list = POI(X_new, model, best_mu)
print(score_list.argmax())

- 라인1: 현재까지 탐색한 모든 해 집합인 X의 평균값을 계산하고 그 가운데 가장 큰값을 best_mu에 저장합니다.
- 라인2: 새로운 100개의 해를 생성하여 X_new에 저장합니다.
- 라인3: POI 함수를 사용해 X_new에 속한 각 해의 점수를 계산 합니다.
- 라인4: argmax 메서드를 이용해 score_list에서 값이 가장 큰 인덱스를 출력합니다.

## 개선 기대 함수
가장 널리 쓰이는 획득 함수인 개선 기대 함수는 새로운 해를 탐색함으로써 기대되는 함숫값의 갱신 정도를 나타냅니다.

가장 널리 쓰이는 획득 함수인 개선 기대 함수는 새로운 해를 탐색함으로써 기대되는 함숫값의 갱신 정도를 나타냅니다.

## 개선 기대 함수 실습
파이썬으로 개선 기대 함수를 구현해보겠습니다.

# 5. 메인 코드

## 메인 코드